In [1]:

import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import seaborn as sns

In [2]:
!make clean && make all

rm -rf *.o
rm main
g++ -o main main.cpp solucion.cpp -O2 -std=c++14


In [3]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

In [4]:
def save_instance(dataset, instance_name, N, impacto, tiempo):
    with open(F"resultados/{dataset}/{instance_name}.txt", "w") as f:
        lista = [N,impacto,tiempo]
        for i in range(len(lista)):
            print(lista[i], file=f, sep=' ', end =' ')

In [5]:
save_instance("Experimento1", "a", 1,0,1);
save_instance("Experimento1", "a", 2,0,1);

In [6]:
CANT_REPETIONES_EXPERIMENTOS = 20

In [7]:
def correr_experimento(metodo, archivo_instancia, cantNodos):
    
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["./main", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    
    
    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.write("\n")
    process.stdin.flush()
   
    #print(instancia)
    
    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise ValueError(F"Hubo un error en la experimentacion para el algoritmo: {metodo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    resultados = []
    tiempo = float(process.stderr.read())
    resultados.append(tiempo)
    
    resultado = process.stdout.read();    
    resultado = resultado.split()
    resultado = map(int, resultado)
    resultado = list(resultado)
  
    resultados += resultado 
    
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return resultados;

In [8]:
archivos= []
run = []
for i in range(31):
    if ( (i > 5) & (i % 2 == 0)):
        archivos.append(f"instancias/CMI_n{i}.in")
        run.append(i)
            

In [9]:

for i in range(len(archivos)):
    print(correr_experimento("M2", archivos[i],run[i]))

ValueError: could not convert string to float: 'Algoritmo no encontrado: M2\nLos algoritmos existentes son: \n\t- H1: Heuristica 1 \n\t- H2: Heuristica 2\n\t- M1C: Metaheuristica colores change\n\t- M1S: Metaheuristica colores swap \n\t- M1SC: Metaheuristica colores swap_y_change\n\t- M1SCH2: Metaheuristica colores swap_y_change con su segunda Heuristica\n\t- M21: Metaheuristica por vertices con heuristica_1\n\t- M22: Metaheuristica por vertices con heuristica_2\n'

# parametros

In [17]:
parametros = ["N", "Impacto", "Tiempo"]


# Experimento 1: Comparacion de heuristicas 


## Heuristica 1

In [11]:

parametros = ["N", "Impacto", "Tiempo"]
filas = []
for i in range((len(archivos))): 
    promedio = 0
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("H1", archivos[i],run[i])
        lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0]]
        promedio+= lista[2]
        
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)
    pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento1/h1.csv", index=False, header=True)           
    


## Heuristica 2

In [12]:

filas = []
for i in range((len(archivos))): 
    promedio = 0
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("H2", archivos[i],run[i])
        lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0]]
        promedio+= lista[2]
        
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)
    pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento1/h2.csv", index=False, header=True)           
    


## Aca se crea el csv de los resultados de la catedra

In [13]:
parametros_sin_tiempo = ["N", "Impacto"]
archivos_catedra= []
run = []
filas = []
for i in range(31):
    if ( (i > 5) & (i % 2 == 0)):
        with open(f"instancias/CMI_n{i}.out") as f:
            c = f.read(2)
            print(c)
            lista = [i,c]
            filas.append(lista)

pd.DataFrame(filas, columns=parametros_sin_tiempo).to_csv("resultados/best_results.csv", index=False, header=True)   

1 
6 
3 
16
12
20
27
25
26
33
38
48
47


# Experimento 3 

## Metaheuristica M2

### Corrido con H2

In [21]:
filas = []
cant_experimentos = len(archivos)*CANT_REPETIONES_EXPERIMENTOS
cant_terminados = 0
barra= "/" 
for i in range((len(archivos))): 
    promedio = 0
    promedio_impacto=0 
    
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("M22", archivos[i],run[i])
        lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0]]
        promedio+= lista[2]
        promedio_impacto+= lista[1]
        cant_terminados+=1
        clear_output(wait=True)
        display('Experimento: ' + str(cant_terminados) + "/" + str(cant_experimentos))
    
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    lista[1] = promedio_impacto/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)
    pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento3/M2H2.csv", index=False, header=True)           


'Experimento: 260/260'

### Corrido con H1

In [18]:
filas = []
cant_experimentos = len(archivos)*CANT_REPETIONES_EXPERIMENTOS
cant_terminados = 0
barra= "/" 
for i in range((len(archivos))): 
    promedio = 0
    promedio_impacto=0 
    
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("M21", archivos[i],run[i])
        lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0]]
        promedio+= lista[2]
        promedio_impacto+= lista[1]
        cant_terminados+=1
        clear_output(wait=True)
        display('Experimento: ' + str(cant_terminados) + "/" + str(cant_experimentos))
    
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    lista[1] = promedio_impacto/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)
    pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento3/M2H1.csv", index=False, header=True)           


'Experimento: 260/260'

## metaheuristica 1

corrido con h1:

In [19]:
filas = []
cant_experimentos = len(archivos)*CANT_REPETIONES_EXPERIMENTOS
cant_terminados = 0
barra= "/" 
for i in range((len(archivos))): 
    promedio = 0
    promedio_impacto=0 
    
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("M1SC", archivos[i],run[i])
        lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0]]
        promedio+= lista[2]
        promedio_impacto+= lista[1]
        cant_terminados+=1
        clear_output(wait=True)
        display('Experimento: ' + str(cant_terminados) + "/" + str(cant_experimentos))
    
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    lista[1] = promedio_impacto/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)
    pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento3/M1H1.csv", index=False, header=True)           


'Experimento: 260/260'

## Heuristica 2

In [20]:
filas = []
cant_experimentos = len(archivos)*CANT_REPETIONES_EXPERIMENTOS
cant_terminados = 0
barra= "/" 
for i in range((len(archivos))): 
    promedio = 0
    promedio_impacto=0 
    
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("M1SCH2", archivos[i],run[i])
        lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0]]
        promedio+= lista[2]
        promedio_impacto+= lista[1]
        cant_terminados+=1
        clear_output(wait=True)
        display('Experimento: ' + str(cant_terminados) + "/" + str(cant_experimentos))
    
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    lista[1] = promedio_impacto/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)
    pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento3/M1H2.csv", index=False, header=True)           


'Experimento: 260/260'

# experimento 3, jugando con las metaheuristicas

In [10]:
def correr_experimento(metodo, archivo_instancia, cantNodos, porcentajeVecindad, iteracionesSinSol):
    
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["./main", metodo, porcentajeVecindad, iteracionesSinSol], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    
    
    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.write("\n")
    process.stdin.flush()
   
    #print(instancia)
    
    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise ValueError(F"Hubo un error en la experimentacion para el algoritmo: {metodo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    resultados = []
    tiempo = float(process.stderr.read())
    resultados.append(tiempo)
    
    resultado = process.stdout.read();    
    resultado = resultado.split()
    resultado = map(int, resultado)
    resultado = list(resultado)
  
    resultados += resultado 
    # print(resultados)
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return resultados;

In [8]:
parametros = ["N", "Impacto", "Tiempo", "porcentajeVecindad", "cantItSinMejoras"]


## Metaheuristica M2

### Corrido con H2

In [14]:
filas = []
filas = []
cant_experimentos = 4 * (10000/500 - 1) * 8 * 20/20
cant_terminados = 0
barra= "/" 
instancia = [1, 5, 9, 12]
porcentajeVecinos = []
listaIteracionesSinMejoras = []
lista = []
CANT_REPETIONES_EXPERIMENTOS = 2
for i in range(500, 10000, 500):
    listaIteracionesSinMejoras.append(i)

t = 0.1
for i in range(8):
    porcentajeVecinos.append(t)
    t += 0.1

In [15]:
 for i in range(len(instancia)):
    for k in range(len(porcentajeVecinos)):
        for t in range(len(listaIteracionesSinMejoras)):
            promedio = 0
            promedio_impacto=0
            for j in range(CANT_REPETIONES_EXPERIMENTOS):
                prueba_resultados = correr_experimento("M22", archivos[instancia[i]], run[i], f"{porcentajeVecinos[k]}",f"{listaIteracionesSinMejoras[t]}")
                lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0], porcentajeVecinos[k], listaIteracionesSinMejoras[t]]
                promedio+= lista[2]
                promedio_impacto+= lista[1]
                cant_terminados+=1
                clear_output(wait=True)
                display('Experimento: ' + str(cant_terminados) + "/" + str(cant_experimentos))
            lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
            lista[1] = promedio_impacto/CANT_REPETIONES_EXPERIMENTOS
            filas.append(lista)

pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento4/M2.csv", index=False, header=True)           


'Experimento: 1216/608.0'

## metaheuristica 1

corrido con h1:

In [13]:
filas = []
cant_experimentos = 4 * (10000/500 - 1) * 8 * 20
cant_terminados = 0
barra= "/" 
instancia = [1, 5, 9, 12]
porcentajeVecinos = []
listaIteracionesSinMejoras = []
lista = []
CANT_REPETIONES_EXPERIMENTOS = 20
for i in range(500, 10000, 500):
    listaIteracionesSinMejoras.append(i)

t = 0.1
for i in range(8):
    porcentajeVecinos.append(t)
    t += 0.1

In [14]:
 for i in range(len(instancia)):
    for k in range(len(porcentajeVecinos)):
        for t in range(len(listaIteracionesSinMejoras)):
            promedio = 0
            promedio_impacto=0
            for j in range(CANT_REPETIONES_EXPERIMENTOS):
                prueba_resultados = correr_experimento("M1SC", archivos[instancia[i]], run[i], f"{porcentajeVecinos[k]}",f"{listaIteracionesSinMejoras[t]}")
                lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0], porcentajeVecinos[k], listaIteracionesSinMejoras[t]]
                promedio+= lista[2]
                promedio_impacto+= lista[1]
                cant_terminados+=1
                clear_output(wait=True)
                display('Experimento: ' + str(cant_terminados) + "/" + str(cant_experimentos))
            lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
            lista[1] = promedio_impacto/CANT_REPETIONES_EXPERIMENTOS
            filas.append(lista)

pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento4/M1.csv", index=False, header=True)           


'Experimento: 12160/12160.0'

# Experimento 4


### Seteo los paramentros que voy a necesitar para corrrer

In [11]:
parametros = ["N", "Impacto", "Tiempo", "densidadG", "densidadH", "nombreExperimento"]

### Cargo los nombres de los archivos, la cantidad de nodos y el nombre del experimento

notar que el experimento i-esimo se corresponde con el archivo i-esimo y la cantidad de nodos i-esima

## m1

In [12]:
archivos= []
names = ['baja', 'media', 'alta']
cantNodos = []
nombreExperimento = []
densidadG = []
densidadH = []
run = []
for i in range(10, 40+1):
    for j in range(len(names)):
        for h in range(len(names)):
            archivos.append(f"nuestrasInstancias/G{names[j]}H{names[h]}{i}.txt")
            cantNodos.append(i)
            nombreExperimento.append(f'G{names[j]}H{names[h]}')
            densidadG.append(f'{names[j]}')
            densidadH.append(f'{names[h]}')
                    

In [15]:
CANT_REPETIONES_EXPERIMENTOS = 10
filas = []
cant_experimentos = len(cantNodos) * CANT_REPETIONES_EXPERIMENTOS
cant_terminados = 0
barra= "/" 
lista = []


In [16]:
 for i in range(len(nombreExperimento)):
    promedio = 0
    promedio_impacto=0
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("M1SC", archivos[i], cantNodos[i], f"{0.5}", f"{6500}")
        lista = [cantNodos[i], prueba_resultados[1], prueba_resultados[0], densidadG[i], densidadH[i], nombreExperimento[i]]
        promedio+= lista[2]
        promedio_impacto+= lista[1]
        cant_terminados+=1
        clear_output(wait=True)
        display('Experimento: ' + str(cant_terminados) + "/" + str(cant_experimentos))
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    lista[1] = promedio_impacto/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)

pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento5/M1.csv", index=False, header=True)           


'Experimento: 10/2790'

KeyboardInterrupt: 

## m2

In [11]:
parametros = ["N", "Impacto", "Tiempo", "densidadG", "densidadH", "nombreExperimento"]

In [12]:
archivos= []
names = ['baja', 'media', 'alta']
cantNodos = []
nombreExperimento = []
densidadG = []
densidadH = []
run = []
for i in range(10, 40+1):
    for j in range(len(names)):
        for h in range(len(names)):
            archivos.append(f"nuestrasInstancias/G{names[j]}H{names[h]}{i}.txt")
            cantNodos.append(i)
            nombreExperimento.append(f'G{names[j]}H{names[h]}')
            densidadG.append(f'{names[j]}')
            densidadH.append(f'{names[h]}')
                    

In [13]:
CANT_REPETIONES_EXPERIMENTOS = 10
filas = []
cant_experimentos = len(cantNodos) * CANT_REPETIONES_EXPERIMENTOS
cant_terminados = 0
barra= "/" 
lista = []


In [14]:
 for i in range(len(nombreExperimento)):
    promedio = 0
    promedio_impacto=0
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("M22", archivos[i], cantNodos[i], f"{0.5}", f"{6500}")
        lista = [cantNodos[i], prueba_resultados[1], prueba_resultados[0], densidadG[i], densidadH[i], nombreExperimento[i]]
        promedio+= lista[2]
        promedio_impacto+= lista[1]
        cant_terminados+=1
        clear_output(wait=True)
        display('Experimento: ' + str(cant_terminados) + "/" + str(cant_experimentos))
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    lista[1] = promedio_impacto/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)

pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento5/M2.csv", index=False, header=True)           


'Experimento: 256/2790'

KeyboardInterrupt: 